<a href="https://colab.research.google.com/github/luciaPardo/DemoGithub/blob/main/AnalisysSentimentModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [5]:
import datetime
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
from google.cloud import aiplatform
!pip3 install tf-models-nightly
!pip3 install tensorflow-text-nightly 
from official.nlp import optimization  # to create AdamW optmizer

#tf.get_logger().setLevel("ERROR")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  0


In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('sentiment_integrado.csv')
df.head(10)

,Unnamed: 0,id,texto,sentimiento,Cantidad palabras,cant caracteres especiales,cant menciones,cant hashtags
0,0,142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",Indefinido,8,4,0,1
1,1,142389933619945473,@PauladeLasHeras No te libraras de ayudar me/n...,Neutral,10,2,1,0
2,2,142391947707940864,@marodriguezb Gracias MAR,Positivo,3,0,1,0
3,3,142416095012339712,"Off pensando en el regalito Sinde, la que se v...",Negativo,24,4,0,0
4,4,142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,Positivo,15,2,0,0
5,5,142424715175280640,"RT @FabHddzC: Si amas a alguien, déjalo libre....",Indefinido,16,6,2,0
6,6,142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,Positivo,8,5,1,0
7,7,142493511634259968,Hoy asisitiré en Madrid a un seminario sobre l...,Indefinido,16,1,0,0
8,8,142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,Positivo,25,2,1,0
9,9,142496796416016384,Desde el escaño. Todo listo para empezar #endi...,Positivo,12,6,0,1


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

/home/azul/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
df["sentimiento"] = df["sentimiento"].apply(lambda x: x if x != "Indefinido" else "Neutral")

In [13]:
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [14]:
mappeo_sent = { "Neutral": 0, "Negativo": 1, "Positivo": 2}
df["num_clasif"] = df["sentimiento"].apply(lambda x: mappeo_sent[x])

In [15]:
texts = df.texto.tolist()
labels = df.num_clasif.tolist()

In [16]:
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [17]:
# Define the BERT model and tokenizer
model_name = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3) # 4 for 4 sentiments


tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 241kB/s]
vocab.txt: 100%|██████████| 872k/872k [00:00<00:00, 2.94MB/s]
tokenizer.json: 100%|██████████| 1.72M/1.72M [00:00<00:00, 2.34MB/s]
config.json: 100%|██████████| 625/625 [00:00<00:00, 2.37MB/s]
model.safetensors: 100%|██████████| 672M/672M [00:31<00:00, 21.6MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Define your dataset instances
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_len=128)

In [19]:
# Define your data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [20]:
# Define training parameters
epochs = 5
lr = 2e-5
optimizer = AdamW(bert_model.parameters(), lr=lr)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = torch.nn.CrossEntropyLoss()

In [22]:
print(torch.cuda.is_available())

False


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to GPU
bert_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [23]:
# Training loop
for epoch in range(epochs):
    bert_model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation loop
    bert_model.eval()
    val_preds = []
    val_labels = []
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = bert_model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)

            val_preds.extend(predicted.tolist())
            val_labels.extend(labels.tolist())

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy}")

Epoch 1/5, Validation Accuracy: 0.6315172817809022
Epoch 2/5, Validation Accuracy: 0.6444053895723492
Epoch 3/5, Validation Accuracy: 0.656414762741652
Epoch 4/5, Validation Accuracy: 0.6537785588752196


KeyboardInterrupt: 